In [79]:
import cv2
import numpy as np
from imutils import contours as imcnts

In [80]:
#variable that will help us to resize the extracted handwritten digits
RESIZED_IMAGE_WIDTH = 30
RESIZED_IMAGE_HEIGHT= 40

In [81]:
#load classifiers on variables
digitClassifications = np.loadtxt("classifications.txt", np.float32)
flatDigitImages = np.loadtxt("flatDigitImages.txt", np.float32)
print(flatDigitImages)

[[  0.   0.   0. ...   0.   0.   0.]
 [  0.   0.   0. ...   0.   0.   0.]
 [  0.   0.   0. ...   0.   0.   0.]
 ...
 [  0.   0.   0. ... 255. 255. 255.]
 [  0.   0.   0. ...   0.   0.   0.]
 [  0.   0.   0. ... 127.  25.   0.]]


In [82]:
#convert digitClassifications into 1-d array (needed for KNN algorithm)
digitClassifications = digitClassifications.reshape((digitClassifications.size, 1))

In [83]:
#creates empty KNearest classifier.
knn = cv2.ml.KNearest_create()
KNearest = knn
knn.train(flatDigitImages, cv2.ml.ROW_SAMPLE, digitClassifications)

True

In [84]:
imgTestSample = cv2.imread("id dataset/test_image/4.jpg")
#th = imgTestSample 
#th=np.invert(th)
bf = cv2.bilateralFilter(imgTestSample, 50, 100, 100) #to remove noise
imgGray = cv2.cvtColor(bf, cv2.COLOR_BGR2GRAY)
# cv2.imshow('test sample',imgGray)
# cv2.waitKey(0)

In [85]:
#retVal, th = cv2.threshold(imgGray, 140, 255, cv2.CHAIN_APPROX_NONE)


th = cv2.adaptiveThreshold(imgGray, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY,7 , 3)
th=np.invert(th)
cv2.imshow('test sample',th)
cv2.waitKey(0)
# opening = cv2.morphologyEx(th, cv2.MORPH_OPEN, np.ones((1, 2), np.uint8))
# cv2.imshow('test sample',opening)
# cv2.waitKey(0)
# dilation = cv2.dilate(opening, np.ones((2, 1), np.uint8),iterations=1)
# cv2.imshow('test sample',dilation)
# cv2.waitKey(0)

-1

In [86]:
# contours, h = cv2.findContours(th, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
# cnt = sorted(contours, key=cv2.contourArea, reverse=True)
# c = cnt[0]
# [intX, intY, intW, intH] = cv2.boundingRect(c)   
# imgDigit = th[intY:intY+intH, intX:intX+intW]
# cv2.imshow('id box',imgDigit)
# cv2.waitKey(0)
imgDigit = th

In [88]:
# cv2.drawContours(imgTestSample, contours, -1, (0,255,0), 3)
# cv2.imshow('test sample',imgTestSample)
# cv2.waitKey(0)
# find the biggest countour (c) by the area
strFinalString = ""
while True:
        contours, h = cv2.findContours(th, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        print(len(contours))
        cnt = sorted(contours, key=cv2.contourArea, reverse=True)
        if(cv2.contourArea(cnt[0])<0.4*imgDigit.shape[0]*imgDigit.shape[1]):
            print('break')
            cv2.imshow('id box',imgDigit)
            cv2.waitKey(0)
            break
        else:
            print('hi')
            c = cnt[0]
            [intX, intY, intW, intH] = cv2.boundingRect(c)   
            imgDigit = imgDigit[intY+5:intY+intH-5, intX+5:intX+intW-5]
            cv2.imshow('id box',imgDigit)
            cv2.waitKey(0)
            
            
cntr, h = cv2.findContours(imgDigit, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cntr,_=imcnts.sort_contours(cntr,method='left-to-right')
print(len(cntr))
for myc in cntr:
    [intX, intY, intW, intH] = cv2.boundingRect(myc)
    imgROI = imgDigit[intY:intY+intH, intX:intX+intW]
    #if(imgROI.shape[0]*imgROI.shape[1] > 100 and imgROI.shape[0]*imgROI.shape[1] < 1000):
    imgROIResized = cv2.resize(imgROI, (RESIZED_IMAGE_WIDTH, RESIZED_IMAGE_HEIGHT))

    cv2.imshow('test sample',imgROI)
    cv2.waitKey(0)

    finalResized = imgROIResized.reshape((1,RESIZED_IMAGE_WIDTH*RESIZED_IMAGE_HEIGHT))
    finalResized = np.float32(finalResized)

    retVal, results, resp, dist =KNearest.findNearest(finalResized, k=1)

    tmpString = str(int(results[0][0]))
    strFinalString = strFinalString + tmpString

print(strFinalString)   


7
break
7
2180255
